In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras 

In [ ]:
train_df=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
train_df

In [ ]:
train_df['Pawpularity'].hist(bins=100)

In [ ]:
train_df[train_df['Pawpularity']==100]

In [ ]:
def append_ext(fn):
    return fn+".jpg"

train_df["Id_path"]=train_df["Id"].apply(append_ext)

In [ ]:
test_df=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test_df["Id_path"]=test_df["Id"].apply(append_ext)
test_df

In [ ]:
pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
image_gen=tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=False, rescale=None,)

In [ ]:
train_generator=image_gen.flow_from_dataframe(dataframe=train_df,
directory="/kaggle/input/petfinder-pawpularity-score/train/",x_col="Id_path",
y_col=
#     'Subject Focus',
#     'Eyes',
#     'Face',
#     'Near',
#     'Action',
#     'Accessory',
#     'Group',
#     'Collage',
#     'Human',
#     'Occlusion',
#     'Info',
#     'Blur',
    "Pawpularity"
,class_mode='raw',batch_size=5)

In [ ]:
test_generator=image_gen.flow_from_dataframe(dataframe=test_df,
directory="/kaggle/input/petfinder-pawpularity-score/test/",x_col="Id_path",
y_col=[]
# #     'Subject Focus',
# #     'Eyes',
# #     'Face',
# #     'Near',
# #     'Action',
# #     'Accessory',
# #     'Group',
# #     'Collage',
# #     'Human',
# #     'Occlusion',
# #     'Info',
# #     'Blur',
#     "Pawpularity"
,class_mode='raw'
#                                              ,batch_size=20
                                            )

In [ ]:
for i in train_generator:
    print(i[0].shape)
    break

In [ ]:

tf.keras.backend.clear_session()
basenet=keras.applications.DenseNet201(include_top=False,input_shape=[256, 256, 3],pooling='avg')
flat1 = keras.layers.Dense(512,activation='relu')(basenet.layers[-1].output)
flat2 = keras.layers.Dense(256,activation='relu')(flat1)
class1 = keras.layers.Dense(1, activation='relu')(flat2)
net=keras.Model(inputs=basenet.inputs,outputs=class1)
loss=keras.losses.MeanSquaredError()
optimizer=keras.optimizers.Adam(1e-4)
metrics=[keras.metrics.MeanAbsoluteError()]
net.compile(loss=loss,optimizer=optimizer,metrics=metrics)

In [ ]:
history=net.fit(train_generator,epochs=15)


In [ ]:
net.save('model.hp5')

In [ ]:
from PIL import Image
base_path='/kaggle/input/petfinder-pawpularity-score/test/'
preds=[]
ids=[]
for i,row in test_df.iterrows():
    
    ad=base_path+row['Id_path']
    img=Image.open(ad).resize((256,256))
    
    img_np=np.expand_dims(np.array(img),0)
    pred=net.predict(img_np)
    ids.append(row['Id'])
    preds.append(pred[0][0])
sub_file=pd.DataFrame({'Id':ids,'Pawpularity':preds})
sub_file

In [ ]:
sub_file.to_csv('submission.csv',index=None)